In [1]:
from copy import deepcopy
import numpy as np


class Reaction:
    
    class _CoeffLaws:
        
        class _BuiltIn:
            def const(**kwargs):
                k = kwargs['k'] if 'k' in kwargs else 1.0
                if k <= 0.0:
                    raise ValueError('k = {0:18.16e}:  Non-positive reaction rate coefficient is prohibited.'.format(k))
                return k
            def arr(**kwargs):
                T = kwargs['T']
                R = kwargs['R'] if 'R' in kwargs else 8.314
                A = kwargs['A'] if 'A' in kwargs else 1.0
                E = kwargs['E'] if 'E' in kwargs else 0.0
                if T <= 0.0:
                    raise ValueError("T = {0:18.16e}:  Non-positive Arrhenius prefactor is prohibited.".format(T))
                if A <= 0.0:
                    raise ValueError("A = {0:18.16e}:  Non-positive temperature is prohibited.".format(A))
                if R <= 0.0:
                    raise ValueError("R = {0:18.16e}:  Non-positive ideal gas constant is prohibited.".format(R))
                return A * np.exp(-E / (R * T))
            def modarr(**kwargs):
                T = kwargs['T']
                R = kwargs['R'] if 'R' in kwargs else 8.314
                A = kwargs['A'] if 'A' in kwargs else 1.0
                b = kwargs['b'] if 'b' in kwargs else 0.0
                E = kwargs['E'] if 'E' in kwargs else 0.0
                if T <= 0.0:
                    raise ValueError("T = {0:18.16e}:  Non-positive Arrhenius prefactor is prohibited.".format(T))
                if A <= 0.0:
                    raise ValueError("A = {0:18.16e}:  Non-positive temperature is prohibited.".format(A))
                if R <= 0.0:
                    raise ValueError("R = {0:18.16e}:  Non-positive ideal gas constant is prohibited.".format(R))
                return A * (T ** b) * np.exp(-E / (R * T))
            
        _dict_builtin = {
            'const' :_BuiltIn.const, 
            'arr'   :_BuiltIn.arr, 
            'modarr':_BuiltIn.modarr
        }
        _dict_full = deepcopy(_dict_builtin)
        
        @classmethod
        def get(cls, name):
            return cls._dict_full[name]
        @classmethod
        def update(cls, name, law):
            if name in cls._dict_builtin:
                raise ValueError('LawName = {}. Changing a built-in law is prohibited.'.format(name))
            cls._dict_full[name] = law
        @classmethod
        def remove(cls, name):
            if name in cls._dict_builtin:
                raise ValueError('LawName = {}. Removing a built-in law is prohibited.'.format(name))
            del cls._dict_full[name]
        @classmethod
        def reset(cls):
            cls._dict_full = deepcopy(cls._dict_builtin)
            
    
    
    def __init__(self, **kwargs):
        self._id          = kwargs['id']          if 'id'          in kwargs else ''
        self._reversible  = kwargs['reversible']  if 'reversible'  in kwargs else False
        self._type        = kwargs['type']        if 'type'        in kwargs else 'elementary'
        self._coeffLaw    = kwargs['coeffLaw']    if 'coeffLaw'    in kwargs else 'const'
        self._coeffParams = kwargs['coeffParams'] if 'coeffParams' in kwargs else {}
        self._reactants   = kwargs['reactants']   if 'reactants'   in kwargs else {}
        self._products    = kwargs['products']    if 'products'    in kwargs else {}
        if self._reversible == True:
            raise NotImplementedError('Reversible reaction is not implemented.')
        if self._type != 'elementary':
            raise NotImplementedError('Type = {}. Non-elementary reaction is not implemented.'.format(self._type))
        if not self._coeffLaw in self._CoeffLaws._dict_full:
            raise NotImplementedError('Law = {}. Refered reaction rate coefficient law is not implemented.'.format(self._coeffLaw))
    
    def rateCoeff(self, **otherParams):
        return self._CoeffLaws._dict_full[self._coeffLaw](**self._coeffParams, **otherParams)
    
    def getReactants(self):
        return self._reactants
    
    def getProducats(self):
        return self._products

In [78]:
import numpy as np

class Reaction_Rate:
    
    def __init__(self, nu_react, nu_prod, concs, k):
        self.nu_react = nu_react
        self.nu_prod = nu_prod
        self.concs = concs
        self.k = k
     
    def progress_rate(self):
        """Returns the progress rate of a system of irreversible, elementary reactions

        INPUTS:
        =======
        nu_react: numpy array of floats, 
                  size: num_species X num_reactions
                  stoichiometric coefficients for the reaction
        k:        array of floats
                  Reaction rate coefficient for the reaction
        concs:    numpy array of floats 
                  concentration of species

        RETURNS:
        ========
        omega: numpy array of floats
               size: num_reactions
               progress rate of each reaction

        EXAMPLES:
        =========
        >>> progress_rate_2(np.array([[2.0, 1.0], [1.0, 0.0], [0.0, 1.0]]), np.array([2.0, 1.0, 1.0]), 10.0)
        array([ 40.,  20.])
        """
        self.progress = self.k # Initialize progress rates with reaction rate coefficients
        for jdx, rj in enumerate(self.progress):
            if rj < 0:
                raise ValueError("k = {0:18.16e}:  Negative reaction rate coefficients are prohibited!".format(rj))
            for idx, xi in enumerate(self.concs):
                nu_ij = self.nu_react[idx,jdx]
                if xi  < 0.0:
                    raise ValueError("x{0} = {1:18.16e}:  Negative concentrations are prohibited!".format(idx, xi))
                if nu_ij < 0:
                    raise ValueError("nu_{0}{1} = {2}:  Negative stoichiometric coefficients are prohibited!".format(idx, jdx, nu_ij))

                self.progress[jdx] *= xi**nu_ij
        return self.progress

    def reaction_rate(self):
        """Returns the reaction rate of a system of irreversible, elementary reactions

        INPUTS:
        =======
        nu_react: numpy array of floats, 
                  size: num_species X num_reactions
                  stoichiometric coefficients for the reactants
        nu_prod:  numpy array of floats, 
                  size: num_species X num_reactions
                  stoichiometric coefficients for the products
        k:        float, default value is 10, 
                  Reaction rate coefficient for the reaction
        concs:    numpy array of floats 
                  concentration of species

        RETURNS:
        ========
        f: numpy array of floats
           size: num_species
           reaction rate of each specie

        EXAMPLES:
        =========
        >>> nu_react = np.array([[1.0, 0.0], [2.0, 0.0], [0.0, 2.0]])
        >>> nu_prod = np.array([[0.0, 1.0], [0.0, 2.0], [1.0, 0.0]])
        >>> r = np.array([ 40.,  20.])
        >>> reaction_rate(nu_react, nu_prod, r)
        array([-20., -40.,   0.])
        """
        nu = self.nu_prod - self.nu_react
        return np.dot(nu, self.progress)

In [79]:
class transform:
    
    def __init__(self, r_ls, element_ls, T):
        num_reaction = len(r_ls)
        self._reactants = np.zeros([len(element_list), num_reaction])
        self._products = np.zeros([len(element_list), num_reaction])
        self._k = np.zeros(num_reaction)
        
        for n, r in enumerate(r_ls):
            self._k[n] = r.rateCoeff(T = T)
            for idx, e in enumerate(element_list):
                self._reactants[idx, n] = r.getReactants()[e] if e in r.getReactants() else 0
                self._products[idx, n] = r.getProducats()[e] if e in r.getProducats() else 0
    
    def rateCoeff(self):
        return self._k
    
    def getReactants(self):
        return self._reactants
    
    def getProducats(self):
        return self._products

In [80]:
r_ls = []
r_ls.append(Reaction(
    reactants=dict(H=1,O2=1), products=dict(OH=1,O=1),
    coeffLaw='arr', coeffParams=dict(E=-8.314)
))

r_ls.append(Reaction(
    reactants=dict(H2=1,O=1), products=dict(OH=1,H=1),
    coeffLaw='arr', coeffParams=dict(E=-8.314)
))

In [81]:
element_list = ['H', 'O', 'OH', 'H2', 'O2']

In [82]:
concentration = np.array([2, 1, 0.5, 1, 1]).transpose()

In [83]:
t = transform(r_ls, element_list, T = 1)

In [84]:
t.rateCoeff()

array([ 2.71828183,  2.71828183])

In [85]:
t.getProducats()

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  1.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [86]:
t.getReactants()

array([[ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  0.],
       [ 0.,  1.],
       [ 1.,  0.]])

In [87]:
reaction_rate = Reaction_Rate(t.getReactants(), t.getProducats(), concentration, t.rateCoeff())

In [89]:
reaction_rate.reaction_rate()

array([-2.71828183,  2.71828183,  8.15484549, -2.71828183, -5.43656366])